In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
df = pd.read_csv('data.csv', sep=',', index_col=0)
df['id'] = df['id'].astype(str)
df['section'] = df['section'].astype(str)
df.head()

,subject,id,section,start_time,end_time,name,attributes,coreqs,prereqs,recurrence,credits
index,,,,,,,,,,,
4,FREN,1001,1,11:00,11:50,Foreign Language (Discovery),NaN,FREN 1001 - 01,NaN,M W F,3
0,HIST,1099,1,09:00,09:50,Social Science (Discovery),NaN,HIST 1099 - 01,NaN,M W F,3
1,HIST,1099,2,10:00,10:50,Social Science (Discovery),NaN,HIST 1099 - 02,NaN,M W F,3
2,PHIL,1000,1,10:00,11:50,Humanities (Discovery),NaN,PHIL 1000 - 01,NaN,Tu Th,4
3,SCI,150,1,10:00,11:50,Science (Discovery),SCI4ALL GLOBALDIV,SCI 150 - 01,NaN,M W F,4


In [3]:
#sort by start time, then by end time
df.sort_values(by=['start_time', 'end_time'], inplace=True)
df['course'] = df['subject'] + ' ' + df['id'] + ':' + df['section']
#convert start and end times to datetime objects
df['start_time'] = pd.to_datetime(df['start_time'])
df['end_time'] = pd.to_datetime(df['end_time'])
#convert recurrences to a list of days
df['recurrence'] = df['recurrence'].str.split(' ')
df.head()


C:\Users\jledl\AppData\Local\Temp\ipykernel_16328\3274769412.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['start_time'] = pd.to_datetime(df['start_time'])
C:\Users\jledl\AppData\Local\Temp\ipykernel_16328\3274769412.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['end_time'] = pd.to_datetime(df['end_time'])


,subject,id,section,start_time,end_time,name,attributes,coreqs,prereqs,recurrence,credits,course
index,,,,,,,,,,,,
0,HIST,1099,1,2024-01-30 09:00:00,2024-01-30 09:50:00,Social Science (Discovery),NaN,HIST 1099 - 01,NaN,"[M, W, F]",3,HIST 1099:1
1,HIST,1099,2,2024-01-30 10:00:00,2024-01-30 10:50:00,Social Science (Discovery),NaN,HIST 1099 - 02,NaN,"[M, W, F]",3,HIST 1099:2
2,PHIL,1000,1,2024-01-30 10:00:00,2024-01-30 11:50:00,Humanities (Discovery),NaN,PHIL 1000 - 01,NaN,"[Tu, Th]",4,PHIL 1000:1
3,SCI,150,1,2024-01-30 10:00:00,2024-01-30 11:50:00,Science (Discovery),SCI4ALL GLOBALDIV,SCI 150 - 01,NaN,"[M, W, F]",4,SCI 150:1
4,FREN,1001,1,2024-01-30 11:00:00,2024-01-30 11:50:00,Foreign Language (Discovery),NaN,FREN 1001 - 01,NaN,"[M, W, F]",3,FREN 1001:1


In [4]:
#find the maximum number of credits without overlap
def create_schedule(df, credit_goal, skip_fridays, start_time, end_time):
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)
    potential_credits = 0
    potential_courses = []
    current_credits = 0
    #sort by start time, then by end time
    df.sort_values(by=['start_time', 'end_time'], inplace=True)
    for index, row in df.iterrows():
        overlap = False
        if current_credits + row['credits'] <= credit_goal:
            for day in row['recurrence']:
                if skip_fridays and day == 'F':
                    overlap = True
                    break
            if row['start_time'] < start_time or row['end_time'] > end_time:
                print('course ' + row['course'] + ' is outside of the time range')
                overlap = True
            #check if the course overlaps with any of the current courses
            for course in potential_courses:
                #check if any of the days overlap
                for day in row['recurrence']:
                    if day in df.loc[df['course'] == course, 'recurrence'].values[0]:
                        if row['start_time'] <= df.loc[df['course'] == course, 'end_time'].values[0]:
                            overlap = True
                            print('overlap between ' + course + ' and ' + row['course'])
                            break
                    else:
                        continue
            if not overlap:
                print('no overlap with any courses, adding ' + row['course'] + ' to potential courses')
                current_credits += row['credits']
                potential_credits = current_credits
                potential_courses.append(row['course'])
        else:
            break
    return potential_credits, potential_courses

print(create_schedule(df, 15, False, '08:00', '17:00'))
    

no overlap with any courses, adding HIST 1099:1 to potential courses
no overlap with any courses, adding HIST 1099:2 to potential courses
no overlap with any courses, adding PHIL 1000:1 to potential courses
overlap between HIST 1099:2 and SCI 150:1
no overlap with any courses, adding FREN 1001:1 to potential courses
(13, ['HIST 1099:1', 'HIST 1099:2', 'PHIL 1000:1', 'FREN 1001:1'])


In [ ]:
#TODO: create a classifying algorithm to find the best schedule
import scikitlearn as sklearn